In [1]:
import os
import json
import jsonlines
from collections import defaultdict
import time

In [2]:
all_start = time.time()

In [3]:
# CMeIE
# A. c→(h,r)
out_name = 'c_to_ht_withtype'
inp_prompt = '''Currently, you are a senior expert in information extraction.
Your task is to extract all possible subject-object entity pairs from the given text. First, extract possible subject spans, and based on the extracted subject spans and the given text, continue to extract the corresponding object spans. Then, identify the entity types of the subject and object from the given entity type list.
The given entity type list is: {entity_list}.
The output format for the task is (subject||subject type||object||object type).
The given text: "{text}"
'''
oup_prompt = '{answer_text}'

In [4]:
data_file_list = [
    'train.json',
    'dev.json',
    'test.json',
]
out_file_list = [
    'train.json',
    'dev.json',
    'test.json'
]

In [5]:
read_dir = './ori/ADE/'
out_dir = './pipeline/ADE'
out_dir = os.path.join(out_dir,out_name)
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
with jsonlines.open(os.path.join(read_dir,'65_schemas.json'),'r') as f:
    trip_types_list = [data for data in f]

In [6]:
trip_types = sorted(set([data['predicate'] for data in trip_types_list]))
ent_type_list = []
for data in trip_types_list:
    ent_type_list.append(data['subject_type'])
    ent_type_list.append(data['object_type'])
ent_types = sorted(set(ent_type_list),reverse=True)

In [7]:
start = time.time()
print('out_name:{}'.format(out_name))
for data_file,out_file in zip(data_file_list,out_file_list):
    read_path = os.path.join(read_dir,data_file)
    out_path = os.path.join(out_dir,out_file)
    with jsonlines.open(read_path,'r') as f:
        datas = [data for data in f]
    with jsonlines.open(out_path,'w') as fw:
        for data in datas:
            inp = inp_prompt.format(relation_list='["' +'","'.join(trip_types) + '"]',entity_list = '["' +'","'.join(ent_types)+ '"]',text=data['text'])
            spo_list = [(item['subject'],item['subject_type'],item['object'], item['object_type']) for item in data['spo_list']]
            processed_spo_list = []
            for spo_item in spo_list:
                if spo_item not in processed_spo_list:
                    processed_spo_list.append(spo_item)
            oup = '\n'.join(['({}||{}||{}||{})'.format(item[0],item[1],item[2],item[3]) for item in processed_spo_list])
            oup = '```\n' + oup.strip() + '\n```'
            out_data = {
                'instruction':inp,
                'input':'',
                'output':oup,
                'text':data['text'],
                'spo_list':data['spo_list']
            }
            fw.write(out_data)
end = time.time()
print('cost:{}秒'.format(round(end-start, 2)))

out_name:c_to_ht_withtype
cost:0.18秒


In [8]:
# CMeIE
# A. c→(r)
out_name = 'c_to_r_withtype'
inp_prompt = '''Currently, you are a senior expert in relation classification.
Your task is to determine the relation type based on the given text from the given relations. The given relation list is: {relation_list}.
The output format for the task is (relation type).
The given text: "{text}"
'''
oup_prompt = '{answer_text}'

In [9]:
data_file_list = [
    'train.json',
    'dev.json',
    'test.json',
]
out_file_list = [
    'train.json',
    'dev.json',
    'test.json'
]

In [10]:
read_dir = './ori/ADE/'
out_dir = './pipeline/ADE'
out_dir = os.path.join(out_dir,out_name)
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
with jsonlines.open(os.path.join(read_dir,'65_schemas.json'),'r') as f:
    trip_types_list = [data for data in f]

In [11]:
trip_types = sorted(set([data['predicate'] for data in trip_types_list]))
ent_type_list = []
for data in trip_types_list:
    ent_type_list.append(data['subject_type'])
    ent_type_list.append(data['object_type'])
ent_types = sorted(set(ent_type_list),reverse=True)

In [12]:
start = time.time()
print('out_name:{}'.format(out_name))
for data_file,out_file in zip(data_file_list,out_file_list):
    read_path = os.path.join(read_dir,data_file)
    out_path = os.path.join(out_dir,out_file)
    with jsonlines.open(read_path,'r') as f:
        datas = [data for data in f]
    with jsonlines.open(out_path,'w') as fw:
        for data in datas:
            inp = inp_prompt.format(relation_list='["' +'","'.join(trip_types) + '"]',entity_list = '["' +'","'.join(ent_types)+ '"]',text=data['text'])
            spo_list = []
            for item in data['spo_list']:
                if item['predicate'] not in spo_list:
                    spo_list.append(item['predicate'])
            processed_spo_list = []
            for spo_item in spo_list:
                if spo_item not in processed_spo_list:
                    processed_spo_list.append(spo_item)
            oup = '\n'.join(['({})a'.format(item) for item in processed_spo_list])
            oup = '```\n' + oup.strip() + '\n```'
            out_data = {
                'instruction':inp,
                'input':'',
                'output':oup,
                'text':data['text'],
                'spo_list':data['spo_list']
            }
            fw.write(out_data)
end = time.time()
print('cost:{}秒'.format(round(end-start, 2)))

out_name:c_to_r_withtype
cost:0.14秒


In [13]:
# CMeIE
# B. c→(t,r)
out_name = 'rc_to_ht_withtype'
inp_prompt = '''Currently, you are a senior expert in information extraction.
Your task is to extract all possible subject-object entity pairs from the given text and relation type. First, extract possible subject spans, and based on the extracted subject spans and the given text, continue to extract the corresponding object spans. Then, identify the entity types of the subject and object from the given entity type list.
The given entity type list is: {entity_list}.
The output format for the task is (subject||subject type||object||object type).
The given text: "{text}"
The given relation type: "{relation}"
'''
oup_prompt = '{answer_text}'

In [14]:
data_file_list = [
    'train.json',
    'dev.json',
    'test.json',
]
out_file_list = [
    'train.json',
    'dev.json',
    'test.json'
]

In [15]:
read_dir = './ori/ADE/'
out_dir = './pipeline/ADE'
out_dir = os.path.join(out_dir,out_name)
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
with jsonlines.open(os.path.join(read_dir,'65_schemas.json'),'r') as f:
    trip_types_list = [data for data in f]

In [16]:
trip_types = sorted(set([data['predicate'] for data in trip_types_list]))

In [17]:
start = time.time()
print('out_name:{}'.format(out_name))
for data_file,out_file in zip(data_file_list,out_file_list):
    read_path = os.path.join(read_dir,data_file)
    out_path = os.path.join(out_dir,out_file)
    with jsonlines.open(read_path,'r') as f:
        datas = [data for data in f]
    with jsonlines.open(out_path,'w') as fw:
        for data in datas:
            relation_to_ht = defaultdict(list)
            for spo_item in data['spo_list']:
                sub = spo_item['subject']
                sub_type = spo_item['subject_type']
                predicate = spo_item['predicate']
                obj = spo_item['object']
                obj_type = spo_item['object_type']
                relation_to_ht[predicate].append((sub, sub_type, obj, obj_type))
            for predicate in relation_to_ht.keys():
                spo_list = relation_to_ht[predicate]
                inp = inp_prompt.format(relation_list='["' +'","'.join(trip_types) + '"]',entity_list = '["' +'","'.join(ent_types)+ '"]',text=data['text'], relation = predicate)
                oup = '\n'.join(['({}||{}||{}||{})'.format(item[0],item[1],item[2],item[3]) for item in spo_list])
                oup = '```\n' + oup.strip() + '\n```'
                out_data = {
                    'instruction':inp,
                    'input':'',
                    'output':oup,
                    'text':data['text'],
                    'spo_list':data['spo_list']
                }
                fw.write(out_data)
end = time.time()
print('cost:{}秒'.format(round(end-start, 2)))

out_name:rc_to_ht_withtype
cost:0.28秒


In [18]:
# CMeIE
# C. h[s1]c→(t,r)
out_name = 'htc_to_r_withtype'
inp_prompt = '''Currently, you are a senior expert in relation classification.
Your task is to determine possible relations from the given relation list based on the given text, subject-object entity pairs, and their types.
The given relation list is: {relation_list}.
The input format for entity pairs is (subject||subject type||object||object type).
The output format for the task is (relation type).
The given text: "{text}"
The given subject-object entity pair: "{head_tail}"
'''
oup_prompt = '{answer_text}'

In [19]:
data_file_list = [
    'train.json',
    'dev.json',
    'test.json',
]
out_file_list = [
    'train.json',
    'dev.json',
    'test.json'
]

In [20]:
read_dir = './ori/ADE/'
out_dir = './pipeline/ADE'
out_dir = os.path.join(out_dir,out_name)
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
with jsonlines.open(os.path.join(read_dir,'65_schemas.json'),'r') as f:
    trip_types_list = [data for data in f]

In [21]:
trip_types = sorted(set([data['predicate'] for data in trip_types_list]))

In [22]:
start = time.time()
print('out_name:{}'.format(out_name))
for data_file,out_file in zip(data_file_list,out_file_list):
    read_path = os.path.join(read_dir,data_file)
    out_path = os.path.join(out_dir,out_file)
    with jsonlines.open(read_path,'r') as f:
        datas = [data for data in f]
    with jsonlines.open(out_path,'w') as fw:
        for data in datas:
            ht_to_relation = defaultdict(list)
            for spo_item in data['spo_list']:
                sub = spo_item['subject']
                sub_type = spo_item['subject_type']
                predicate = spo_item['predicate']
                obj = spo_item['object']
                obj_type = spo_item['object_type']
                ht_item = (sub, sub_type, obj, obj_type)
                ht_to_relation[ht_item].append(predicate)
            for ht_item in ht_to_relation.keys():
                predicate = ht_to_relation[ht_item]
                ht_item_str = '({}||{}||{}||{})'.format(ht_item[0], ht_item[1], ht_item[2], ht_item[3])
                inp = inp_prompt.format(relation_list='["' +'","'.join(trip_types) + '"]',entity_list = '["' +'","'.join(ent_types)+ '"]',text=data['text'], head_tail = ht_item_str)
                oup = '\n'.join(['({})'.format(item) for item in predicate])
                oup = '```\n' + oup.strip() + '\n```'
                out_data = {
                    'instruction':inp,
                    'input':'',
                    'output':oup,
                    'text':data['text'],
                    'spo_list':data['spo_list']
                }
                fw.write(out_data)

end = time.time()
print('cost:{}秒'.format(round(end-start, 2)))

out_name:htc_to_r_withtype
cost:0.26秒


In [23]:
# CMeIE
# A0. c→(h,r,t)
out_name = 'c_to_hrt_withtype'
inp_prompt = '''Currently, you are a senior expert in information extraction. 
Your task is to extract relational triplets from the given text. First, extract the subject-object entity pairs from the given text, and based on these subject-object entity pairs, identify the subject entity types and the object entity types from the given entity type list. The given entity type  list is: {entity_list}.
Then, based on the subject-object entity pairs and their corresponding entity types, extract the possible relations from the given relation list. The given relation list is: {relation_list}.
The output format for the task is (subject||subject type||relation||object||object type).
The given text:"{text}"
'''
oup_prompt = '{answer_text}'

In [24]:
data_file_list = [
    'train.json',
    'dev.json',
    'test.json',
]
out_file_list = [
    'train.json',
    'dev.json',
    'test.json'
]

In [25]:
read_dir = './ori/ADE/'
out_dir = './pipeline/ADE'
out_dir = os.path.join(out_dir,out_name)
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
with jsonlines.open(os.path.join(read_dir,'65_schemas.json'),'r') as f:
    trip_types_list = [data for data in f]

In [26]:
trip_types = sorted(set([data['predicate'] for data in trip_types_list]))

In [27]:
start = time.time()
print('out_name:{}'.format(out_name))
for data_file,out_file in zip(data_file_list,out_file_list):
    read_path = os.path.join(read_dir,data_file)
    out_path = os.path.join(out_dir,out_file)
    with jsonlines.open(read_path,'r') as f:
        datas = [data for data in f]
    with jsonlines.open(out_path,'w') as fw:
        for data in datas:
            inp = inp_prompt.format(relation_list='["' +'","'.join(trip_types) + '"]',entity_list = '["' +'","'.join(ent_types)+ '"]',text=data['text'])
            spo_list = [(item['subject'],item['subject_type'],item['predicate'],item['object'],item['object_type']) for item in data['spo_list']]
            processed_spo_list = []
            for spo_item in spo_list:
                if spo_item not in processed_spo_list:
                    processed_spo_list.append(spo_item)
            oup = '\n'.join(['({}||{}||{}||{}||{})'.format(item[0],item[1],item[2],item[3],item[4]) for item in processed_spo_list])
            oup = '```\n' + oup.strip() + '\n```'
            out_data = {
                'instruction':inp,
                'input':'',
                'output':oup,
                'text':data['text'],
                'spo_list':data['spo_list']
            }
            fw.write(out_data)
end = time.time()
print('cost:{}秒'.format(round(end-start, 2)))

out_name:c_to_hrt_withtype
cost:0.22秒


In [28]:
all_end = time.time()
all_end-all_start

24.899449586868286